-------------

# Imports

In [1]:

from PIL import Image  # To transform the image in the Processor
import numpy as np
import gym

# Convolutional Backbone Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam

# Keras-RL
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from tensorflow.keras.models import load_model


2023-01-23 20:27:40.091380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
env = gym.make("poison_triple:poison_triple_v0")
nb_actions = env.action_space.n


-----
# Image Processing

In [3]:
nb_actions

4

In [4]:
IMG_SHAPE = (84, 84)
WINDOW_LENGTH = 4


In [5]:
class ImageProcessor(Processor):
    def process_observation(self, observation):
        # First convert the numpy array to a PIL Image
        img = Image.fromarray(observation)
        # Then resize the image
        img = img.resize(IMG_SHAPE)
        # And convert it to grayscale  (The L stands for luminance)
        img = img.convert("L")
        # Convert the image back to a numpy array and finally return the image
        img = np.array(img)
        return img.astype('uint8')  # saves storage in experience memory
    
    def process_state_batch(self, batch):

        # We divide the observations by 255 to compress it into the intervall [0, 1].
        # This supports the training of the network
        # We perform this operation here to save memory.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## Model Creation

**NOTE: Depending on your custom environment, this model will vary greatly, try reading papers that are solving similar problems to your own!**

In [6]:
input_shape = (WINDOW_LENGTH, IMG_SHAPE[0], IMG_SHAPE[1])
input_shape

(4, 84, 84)

In [7]:
# MODEL LOADING

#model = load_model("test_dqn_snake_weights_100000.h5f")

# MODEL FROM SCRATCH
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))

model.add(Convolution2D(32, (8, 8), strides=(4, 4),kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

#print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 7, 7, 64)          0

----
## Creating the Agent

In [8]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)


In [9]:
processor = ImageProcessor()


In [10]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)


In [11]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
              train_interval=4, delta_clip=1)

In [12]:
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


2023-01-23 20:27:42.434802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 20:27:42.435911: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-23 20:27:42.442398: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [13]:
weights_filename = 'dqn_poison_tri_weights.h5f'
checkpoint_weights_filename = 'dqn_' + "poison_tri" + '_weights_{step}.h5f'
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_weights_filename, interval=100000)


In [14]:
#dqn.fit(env, nb_steps=1200000, callbacks=[checkpoint_callback], log_interval=100000, visualize=False)

# After training is done, we save the final weights one more time.
#dqn.save_weights(weights_filename, overwrite=True)



In [15]:
# Load the weights
model.load_weights("colab_weights/1_20_2023/dqn_poison_tri_weights_1200000.h5f")


memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=.1, value_test=.05,
                              nb_steps=100000)

processor = ImageProcessor()

# Initialize the DQNAgent with the new model and updated policy and compile it
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


In [16]:
env.sleep = 0.2

In [17]:
dqn.test(env, nb_episodes=20, visualize=True)

Testing for 20 episodes ...
EATING
WHY?
WHY?


/home/wlw/anaconda3/envs/snake/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/wlw/anaconda3/envs/snake/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/wlw/anaconda3/envs/snake/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/home/wlw/anaconda3/envs/snake/lib/python3.10/site-packages/keras/engine/training_v1.py

EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
Episode 1: reward: 4.000, steps: 47
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
Episode 2: reward: 3.000, steps: 34
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
Episode 3: reward: 5.000, steps: 167
WHY?
EATING
WHY?
EATING
WHY?
Episode 4: reward: 1.000, steps: 1001
EATING
WHY?
EATING
WHY?
EATING
WHY?
WHY?
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
Episode 5: reward: 7.000, steps: 216
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
Episode 6: reward: 7.000, steps: 179
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
WHY?
EATING
WHY?
Episode 7: reward: 3.000, steps: 77
EATING
WHY?
EATING
WHY?
EATING
WHY?
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?
EATING
WHY?


KeyboardInterrupt: 